In [1]:
import requests
from bs4 import BeautifulSoup
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_chroma import Chroma
from langchain_core.prompts import ChatPromptTemplate
from langchain_ollama.llms import OllamaLLM
from langchain.chains import RetrievalQA
from langchain.docstore.document import Document

In [2]:
def extract_text_from_url(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    return soup.get_text()

In [3]:
def split_text(text, chunk_size=500):
    return [text[i:i + chunk_size] for i in range(0, len(text), chunk_size)]


In [22]:
llm = OllamaLLM(model="llama3.2:1b", server_url="http://localhost:11434")  # Ajusta el puerto si es necesario


In [23]:
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

In [24]:
vectorstore = Chroma(persist_directory="./vectorstore", embedding_function=embedding_model)

In [25]:
url = "https://english.elpais.com/usa/2024-12-15/the-border-biden-leaves-behind-stable-thanks-to-mexico-and-doors-closed-to-asylum-seekers.html"
page_content = extract_text_from_url(url)

In [26]:
chunks = split_text(page_content)
print(f"Number of chunks: {len(chunks)}")

Number of chunks: 32


In [27]:
documents = [Document(page_content=chunk) for chunk in chunks]
vectorstore.add_documents(documents)
print("Documents added to the vector store.")

Documents added to the vector store.


In [28]:
prompt = ChatPromptTemplate.from_template(
    template="You are a helpful assistant. Use the context below to answer the user's question:\n\n{context}\n\nQuestion: {question}\nAnswer:"
)

In [29]:
retriever = vectorstore.as_retriever()

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    chain_type_kwargs={"prompt": prompt}
)

In [31]:
query = "What is the main topic of the webpage?"
response = qa_chain.run(query)
print("Response:", response)

ResponseError: model requires more system memory (2.1 GiB) than is available (2.0 GiB)